In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [13]:
#Query data for getting the gross number of captains
q_captain = """
select pickup_location_hex_8, yyyymmdd, order_type, time, count(distinct captain_id) as gross_captains from 
(select 
        order_id, updated_epoch, yyyymmdd, pickup_location_hex_8, captain_location_hex_8, drop_location_hex_8 as hex_id,
        city_name, order_type, captain_id, event_type, order_status, hhmmss as time,
        cardinality(array_distinct(array_remove(split(replace(replace(replace(map_riders, '['), ']'), '"'), ','), ''))) as map_riders_count,
        array_agg(event_type) over (partition by order_id) as event_type_agg, updated_hhmmss
        
from 
        orders.order_logs_immutable
where 
        yyyymmdd >= '20220919'
        and yyyymmdd <= '20221016'
        and pickup_location_hex_8 IN ('883da106c1fffff', '883da106c9fffff', '883da1a939fffff', '883da1a93dfffff', '883da106c3fffff', '883da106c5fffff',
         '883da106c7fffff', '883da106cbfffff', '883da106cdfffff', '883da1a903fffff', '883da1a907fffff', '883da1a915fffff', '883da1a923fffff', '883da1a927fffff',
          '883da1a92bfffff', '883da1a931fffff', '883da1a935fffff', '883da1a93bfffff') and order_type IN ('app'))
group by 1, 2, 3, 4
"""

df_captain_copy = pd.read_sql(q_captain, connection)

In [17]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta
0,6348918dd9dc7714195006a3,1.873,Delhi,883da11185fffff,28.508549,77.088864,040037,04,0400,Link,04:00,5,20221014,dropped,Asked to send parcel,6.0
1,634893b4550449506931bca3,4.676,Delhi,883da1a939fffff,28.627647,77.375916,040948,04,0400,Link,04:00,5,20221014,dropped,None,10.0
2,634898db19caea40e13cf788,0.253,Delhi,883da1145dfffff,28.643122,77.218254,043147,04,0430,Link,04:59,5,20221014,dropped,None,6.0
3,634893f919caea40e13cf5fe,1.436,Delhi,883da10647fffff,28.588299,77.314705,041057,04,0400,Link,04:00,5,20221014,customerCancelled,Change of plans,3.0
4,634895dda51bb634fb941fde,NaN,Delhi,883da18cabfffff,28.701694,77.147903,041901,04,0415,Link,04:00,5,20221014,customerCancelled,Order cancelled before rider was mapped,0.0


In [18]:
#View the dataset
df_captain = df_captain_copy.copy()
df_captain.head()

,pickup_location_hex_8,yyyymmdd,order_type,time,gross_captains
0,883da1a923fffff,20221007,app,214501,3
1,883da106cbfffff,20221007,app,181707,5
2,883da106c3fffff,20221006,app,203136,4
3,883da106c9fffff,20221006,app,101241,6
4,883da1a93bfffff,20220924,app,153924,2


In [20]:
#Load datasets including hexes from level_1 and level_2
df_level_1 = pd.read_csv('delhi_location_data_level1_demand_zone2.csv')

level1_hexes = list(df_level_1['customer_location_hex_8'].unique())

#Filter only on the hexes needed
df_filter_level1 = df[df['customer_location_hex_8'].isin(level1_hexes)]
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].apply(lambda x: x[6:])

In [21]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA'))

In [22]:
#Zone-level-metrics, calculated on per hour basis

hourly_metrics = []
date_metrics = {}
for date in df_filter_level1['date'].unique().tolist():
    df_date = df_filter_level1[df_filter_level1['date'] == date]
    for hour in list(df_date['hour'].unique()):
        df_hourly_level1 = df_date[df_date['hour'] == hour]

        #OCARA
        df_cancel_level1 = df_hourly_level1[df_hourly_level1['order_status'] == 'customerCancelled']

        ocara_zone_level1 = round((df_cancel_level1[df_cancel_level1['new_cancel_reason'] == 'OCARA'].shape[0]/df_hourly_level1.shape[0]), 4)
        #COBRA
        cobra_zone_level1 = round((df_cancel_level1[df_cancel_level1['new_cancel_reason'] == 'COBRA'].shape[0]/df_hourly_level1.shape[0]), 4)

        #COBRM
        cobrm_zone_level1 = round((df_cancel_level1[df_cancel_level1['new_cancel_reason'] == 'COBRM'].shape[0]/df_hourly_level1.shape[0]), 4)

        #G2N
        g2n_zone_level1 = round((df_hourly_level1[df_hourly_level1['order_status'] == 'dropped'].shape[0]/df_hourly_level1.shape[0]), 4)

        #ETA
        eta_zone_level1 = round(df_hourly_level1['eta'].mean(), 2)

        #FM
        dropped_rides_level_1 = df_hourly_level1[df_hourly_level1['order_status'] == 'dropped']
        fm_zone_level1 = round(dropped_rides_level_1['accept_to_pickup_distance'].mean(), 2)

        #Create list of different metrics
        level1_metrics = [date, hour, ocara_zone_level1, cobra_zone_level1, cobrm_zone_level1, g2n_zone_level1, eta_zone_level1, fm_zone_level1]
        hourly_metrics.append(level1_metrics)
metrics_df = pd.DataFrame(hourly_metrics)
metrics_df.columns = ['date', 'hour', 'OCARA', 'COBRA', 'COBRM', 'G2N', 'ETA', 'FM']
metrics_df = metrics_df.sort_values(by = ['date', 'hour'], ascending=True)
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM
662,01,00,0.3704,0.1296,0.0000,0.5000,3.00,0.69
669,01,01,0.4242,0.0909,0.0000,0.4848,2.97,1.02
653,01,02,0.3226,0.1935,0.0000,0.4839,2.52,0.45
663,01,03,0.3966,0.1897,0.0000,0.4138,2.84,1.10
664,01,04,0.3409,0.1023,0.0000,0.5568,3.22,0.91
...,...,...,...,...,...,...,...,...
533,30,19,0.1926,0.1278,0.0026,0.6664,2.37,0.49
534,30,20,0.2009,0.1065,0.0046,0.6819,2.43,0.44
531,30,21,0.2151,0.0959,0.0029,0.6860,2.42,0.42
550,30,22,0.2489,0.1135,0.0000,0.6376,2.88,0.55


In [23]:
#Get the number of rides by order status
rides_by_order = pd.pivot_table(df_filter_level1, index = ['date', 'hour'], columns = 'order_status', values = 'count', aggfunc = 'sum',
 fill_value=0).reset_index().rename_axis(None, axis=1)
rides_by_order['total'] = rides_by_order[['aborted', 'customerCancelled', 'dropped', 'expired']].sum(axis = 1)
rides_by_order['Expiry %'] = round(rides_by_order['expired']/rides_by_order['total'], 4)

#Merge with metrics
metrics_df = metrics_df.merge(rides_by_order[['date', 'hour', 'Expiry %', 'total']], on = ['date', 'hour'], how = 'left')
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total
0,01,00,0.3704,0.1296,0.0000,0.5000,3.00,0.69,0.0000,54
1,01,01,0.4242,0.0909,0.0000,0.4848,2.97,1.02,0.0000,33
2,01,02,0.3226,0.1935,0.0000,0.4839,2.52,0.45,0.0000,31
3,01,03,0.3966,0.1897,0.0000,0.4138,2.84,1.10,0.0000,58
4,01,04,0.3409,0.1023,0.0000,0.5568,3.22,0.91,0.0000,88
...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.1926,0.1278,0.0026,0.6664,2.37,0.49,0.0105,1142
668,30,20,0.2009,0.1065,0.0046,0.6819,2.43,0.44,0.0046,657
669,30,21,0.2151,0.0959,0.0029,0.6860,2.42,0.42,0.0000,344
670,30,22,0.2489,0.1135,0.0000,0.6376,2.88,0.55,0.0000,229


In [24]:
#Create a column for hour, minute and second in df_captain
df_captain['hour'] = df_captain['time'].apply(lambda x: x[0:2])
df_captain['minute'] = df_captain['time'].apply(lambda x: x[2:4])
df_captain['second'] = df_captain['time'].apply(lambda x: x[4:])
df_captain['date'] = df_captain['yyyymmdd'].apply(lambda x: x[6:])

#Create a column for getting the 20_sec_batch
df_captain['minute'] = df_captain['minute'].astype(float)
df_captain['second'] = df_captain['second'].astype(float)
df_captain['20_sec_batch'] = pd.cut(df_captain['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df_captain['count'] = 1
df_captain.head()

,pickup_location_hex_8,yyyymmdd,order_type,time,gross_captains,hour,minute,second,date,20_sec_batch,count
0,883da1a923fffff,20221007,app,214501,3,21,45.0,1.0,07,0-20,1
1,883da106cbfffff,20221007,app,181707,5,18,17.0,7.0,07,0-20,1
2,883da106c3fffff,20221006,app,203136,4,20,31.0,36.0,06,20-40,1
3,883da106c9fffff,20221006,app,101241,6,10,12.0,41.0,06,40-60,1
4,883da1a93bfffff,20220924,app,153924,2,15,39.0,24.0,24,20-40,1


In [25]:
#Get the number of captains coming in each hour and every 20_sec_batch
captain_hourly = df_captain.groupby(['date', 'hour'])['gross_captains'].sum().reset_index()
captain_hourly = captain_hourly.rename(columns={'gross_captains': 'hourly_gross_captains'})

captain_20_sec_batch = df_captain.groupby(['date', 'hour', 'minute', '20_sec_batch'])['gross_captains'].sum().reset_index()

#Get the average number of captains available in each 20_sec_batch per hour
captain_20_sec_average = captain_20_sec_batch.groupby(['date', 'hour'])['gross_captains'].mean().reset_index()
captain_20_sec_average['gross_captains'] = captain_20_sec_average['gross_captains'].apply(lambda x: round(x, 2))
captain_20_sec_average = captain_20_sec_average.rename(columns= {'gross_captains':'20_sec_gross_captains'})

#Merge the hourly captains and 20_sec_captains with the metrics table
metrics_df = metrics_df.merge(captain_hourly, on = ['date', 'hour'], how = 'left')
metrics_df = metrics_df.merge(captain_20_sec_average, on = ['date', 'hour'], how = 'left')
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains
0,01,00,0.3704,0.1296,0.0000,0.5000,3.00,0.69,0.0000,54,165,0.92
1,01,01,0.4242,0.0909,0.0000,0.4848,2.97,1.02,0.0000,33,108,0.60
2,01,02,0.3226,0.1935,0.0000,0.4839,2.52,0.45,0.0000,31,108,0.60
3,01,03,0.3966,0.1897,0.0000,0.4138,2.84,1.10,0.0000,58,197,1.09
4,01,04,0.3409,0.1023,0.0000,0.5568,3.22,0.91,0.0000,88,312,1.73
...,...,...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.1926,0.1278,0.0026,0.6664,2.37,0.49,0.0105,1142,4150,23.06
668,30,20,0.2009,0.1065,0.0046,0.6819,2.43,0.44,0.0046,657,2025,11.25
669,30,21,0.2151,0.0959,0.0029,0.6860,2.42,0.42,0.0000,344,905,5.03
670,30,22,0.2489,0.1135,0.0000,0.6376,2.88,0.55,0.0000,229,662,3.68


In [26]:
#Load the deamnd_summary and add it to the metrics
df_demand = pd.read_csv('demand_delhi.csv')
df_demand = df_demand.sort_values(by = ['date','hour'], ascending = True).reset_index(drop = True)
metrics_df['rides_per_hour'] = df_demand['rides_per_hour']
metrics_df['rides_per_20_sec'] = df_demand['rides_per_20_sec']
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains,rides_per_hour,rides_per_20_sec
0,01,00,0.3704,0.1296,0.0000,0.5000,3.00,0.69,0.0000,54,165,0.92,18.0,0.10
1,01,01,0.4242,0.0909,0.0000,0.4848,2.97,1.02,0.0000,33,108,0.60,11.0,0.06
2,01,02,0.3226,0.1935,0.0000,0.4839,2.52,0.45,0.0000,31,108,0.60,10.0,0.06
3,01,03,0.3966,0.1897,0.0000,0.4138,2.84,1.10,0.0000,58,197,1.09,19.0,0.11
4,01,04,0.3409,0.1023,0.0000,0.5568,3.22,0.91,0.0000,88,312,1.73,29.0,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.1926,0.1278,0.0026,0.6664,2.37,0.49,0.0105,1142,4150,23.06,381.0,2.12
668,30,20,0.2009,0.1065,0.0046,0.6819,2.43,0.44,0.0046,657,2025,11.25,219.0,1.22
669,30,21,0.2151,0.0959,0.0029,0.6860,2.42,0.42,0.0000,344,905,5.03,115.0,0.64
670,30,22,0.2489,0.1135,0.0000,0.6376,2.88,0.55,0.0000,229,662,3.68,76.0,0.42


In [27]:
#Load the decrease in FM due to batching and add it to the metrics
decrease_FM = pd.read_excel('decrease_FM_batching2.xlsx')
decrease_FM = decrease_FM.sort_values(by = ['date', 'hour'], ascending = True).reset_index(drop = True)
metrics_df['FM_decrease'] = decrease_FM['FM_decrease']
metrics_df

,date,hour,OCARA,COBRA,COBRM,G2N,ETA,FM,Expiry %,total,hourly_gross_captains,20_sec_gross_captains,rides_per_hour,rides_per_20_sec,FM_decrease
0,01,00,0.3704,0.1296,0.0000,0.5000,3.00,0.69,0.0000,54,165,0.92,18.0,0.10,0.0208
1,01,01,0.4242,0.0909,0.0000,0.4848,2.97,1.02,0.0000,33,108,0.60,11.0,0.06,-0.0675
2,01,02,0.3226,0.1935,0.0000,0.4839,2.52,0.45,0.0000,31,108,0.60,10.0,0.06,-0.0616
3,01,03,0.3966,0.1897,0.0000,0.4138,2.84,1.10,0.0000,58,197,1.09,19.0,0.11,0.0205
4,01,04,0.3409,0.1023,0.0000,0.5568,3.22,0.91,0.0000,88,312,1.73,29.0,0.16,-0.0574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,30,19,0.1926,0.1278,0.0026,0.6664,2.37,0.49,0.0105,1142,4150,23.06,381.0,2.12,-0.0650
668,30,20,0.2009,0.1065,0.0046,0.6819,2.43,0.44,0.0046,657,2025,11.25,219.0,1.22,-0.0441
669,30,21,0.2151,0.0959,0.0029,0.6860,2.42,0.42,0.0000,344,905,5.03,115.0,0.64,-0.0525
670,30,22,0.2489,0.1135,0.0000,0.6376,2.88,0.55,0.0000,229,662,3.68,76.0,0.42,-0.0024


In [28]:
metrics_df.to_csv('metrics_delhi2.csv', index = False)